# ![HELP LOGO](https://avatars1.githubusercontent.com/u/7880370?s=100&v=4>)VO Simple Examples

The data available from the HELP consortium is available through the HELP Virtual Observatory Server. There are several catalogues available and basic queries can be made through a HTML form, using ADQL/SQL. For more complex queries, we recommend the use of pyvo. What follows are some examples using this module, with a specific focus on the CIGALE results for the COSMOS and ELAIS-N1 field.

Alternatively, head to HeDaM and download the full fits files directly. The link can be found in the main 'README' of the github.

### pyvo - Download and installation

Try this website: https://pyvo.readthedocs.io/en/latest/ 

You should just be able to install using pip.

### Examples

First, we must set up the service object :

In [2]:
# Import the module
import pyvo as vo
# Setting up the service object 
service = vo.dal.TAPService("http://vohedamtest.lam.fr/__system__/tap/run/tap")

Then we can try downloading the data. Queries are sent using the ADQL language, which is based on SQL. Some useful links for basic queries:

https://www.gaia.ac.uk/data/gaia-data-release-1/adql-cookbook
http://tapvizier.u-strasbg.fr/adql/help.html

There are certain input fields and output fields that work specifically with the HELP datasets, which can be found here:

http://vohedamtest.lam.fr/herschelhelp/q/cone/info


The query below selects 10 sources from the main HELP catalogue within a defined search radius within the COSMOS field (as defined by the chosen RA and Dec). The result displays their RA, Dec and stellar masses from CIGALE:

In [115]:
# Downloading the data
resultset = service.search(
    "SELECT TOP 10 ra, dec, cigale_mstar FROM herschelhelp.main WHERE 1=CONTAINS(POINT('ICRS', ra, dec)," +
    "CIRCLE('ICRS',"+str(150.0)+", "+str(2.35)+", 0.025 ))"
)


Various information is encoded within the downloaded table: 

In [140]:
keys = resultset.infos.keys()
print(" - Keys: ", keys)
server = resultset.infos['server']
print(" - Server used to download the data: ", server)
query = resultset.infos['query']
print(" - The query sent - this includes details of all the columns returned, which is particularly useful if * is used instead of specifying columns: ", query)
src = resultset.infos['src_res']
print(" - Resource inforation: ", src)
table = resultset.infos['src_table']
print(" - Information on the source table: ", table)
yesno = resultset.infos['QUERY_STATUS']
print(" - Did the query work? ", yesno)
cita = resultset.infos['citation']
print(" - Citation: ", cita)

 - Keys:  dict_keys(['server', 'query', 'src_res', 'src_table', 'QUERY_STATUS', 'citation'])
 - Server used to download the data:  http://vohedamtest.lam.fr
 - The query sent - this includes details of all the columns returned, which is particularly useful if * is used instead of specifying columns:  SELECT ra, dec, cigale_mstar FROM herschelhelp.main WHERE herschelhelp.main.field = 'ELAIS-N1' AND herschelhelp.main.cigale_mstar IS NOT NULL LIMIT 1000000
 - Resource inforation:  Contains traces from resource herschelhelp/q
 - Information on the source table:  Contains traces from table herschelhelp.main
 - Did the query work?  OK
 - Citation:  http://vohedamtest.lam.fr/tableinfo/herschelhelp.main


The resulting table can be accessed as an Astropy table:

In [121]:
mytable = resultset.table
cols = resultset.table.colnames
print(mytable)
print(cols)

        ra                dec         cigale_mstar
       deg                deg             Msun    
------------------ ------------------ ------------
  149.985827541585 2.3295548291189498           --
149.98517171899101   2.33014869158335           --
149.98567771899101   2.33002169158335           --
  149.986950399064 2.3293979761682801           --
  149.987236718991 2.3301936915833501           --
149.98178171899099   2.33379869158335           --
  149.982222897837 2.3336755537237601           --
149.98033296301699 2.3359708718071799           --
  149.981787718991 2.3350976915833499           --
  149.982416718991 2.3360796915833499           --
['ra', 'dec', 'cigale_mstar']


Note that the 'cigale_mstar' column is empty. This is fine, not all sources have CIGALE results. Below are some examples of some different queries, including an example to deal with empty rows:

In [125]:
# Return the data for 20 sources in the COSMOS field: all available columns, even if these do not contain values.
resultset = service.search("SELECT TOP 20 * FROM herschelhelp.main WHERE herschelhelp.main.field='COSMOS'")
resultset.table

<Table masked=True length=20>
field            help_id           ... flag_optnir_obs flag_optnir_det
                                   ...                                
object            object           ...      int16           int16     
------ --------------------------- ... --------------- ---------------
COSMOS HELP_J095711.278+005908.932 ...               3               1
COSMOS HELP_J095711.970+005902.678 ...               3               1
COSMOS HELP_J095711.507+005906.828 ...               3               1
COSMOS HELP_J095711.636+005907.461 ...               3               1
COSMOS HELP_J095711.752+005906.949 ...               3               1
COSMOS HELP_J095711.438+005913.977 ...               3               1
COSMOS HELP_J095710.894+005919.458 ...               3               1
COSMOS HELP_J095711.154+005919.006 ...               3               0
COSMOS HELP_J095712.057+005913.381 ...               3               0
COSMOS HELP_J095710.884+005924.242 ...               3               1
COSMOS HELP_J095711.252+005920.998 ...               3               1
COSMOS HELP_J095711.200+005928.496 ...               3               1
COSMOS HELP_J095711.360+005927.398 ...               3               1
COSMOS HELP_J095711.762+005924.884 ...               3               1
COSMOS HELP_J095711.842+005925.086 ...               3               1
COSMOS HELP_J095711.675+005926.448 ...               3               1
COSMOS HELP_J095711.689+005931.778 ...               3               1
COSMOS HELP_J095710.463+005951.881 ...               3               1
COSMOS HELP_J095710.792+005951.481 ...               3               1
COSMOS HELP_J095711.649+005933.913 ...               3               1

In [126]:
# Return the ra, dec and cigale dust luminosities for 10 sources in the field ELAIS-N1
resultset = service.search("SELECT TOP 10 ra, dec, cigale_dustlumin FROM herschelhelp.main WHERE herschelhelp.main.field='ELAIS-N1'")
resultset.table

ra,dec,cigale_dustlumin
deg,deg,W
float64,float64,float64
246.99914718805701,55.113613018499301,--
246.997735636057,55.114437479499301,--
246.99983216905699,55.116273360499299,--
247.02517817505699,55.164718342499299,--
247.019958437057,55.151093486499299,--
247.01915320505699,55.152713321499299,--
247.018785255057,55.152659522499299,--
247.01847224405699,55.152491255499299,--


In [132]:
# Return the ra, dec and cigale stellar masses for 200 sources in the field ELAIS-N1 where mstar has a value
search1 = "SELECT ra, dec, cigale_mstar FROM "
search2 = "herschelhelp.main WHERE herschelhelp.main.field='ELAIS-N1' " 
search3 = "AND herschelhelp.main.cigale_mstar IS NOT NULL " 

resultset = service.search(search1 + search2 + search3)
resultset.table #.show_in_notebook()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


Note that the data in the columns is masked, but can be treated as a normal Astropy table for plotting etc. To end this example notebook, I just save the table. I encourage you to examine other notebooks in this folder for examples on how to make use of the data provided by HELP.

In [139]:
resultset.table.write('CIGALE_VOTable.fits', overwrite=True)

 ![HELP LOGO](https://avatars1.githubusercontent.com/u/7880370?s=75&v=4)

**Authors**: [Jenifer Millard](http://www.cardiff.ac.uk/people/research-students/view/914013-millard-jenifer)

 
For a full description of the database and how it is organised in to `dmu_products` please the top level [readme](../readme.md).
 
The Herschel Extragalactic Legacy Project, ([HELP](http://herschel.sussex.ac.uk/)), is a [European Commission Research Executive Agency](https://ec.europa.eu/info/departments/research-executive-agency_en)
funded project under the SP1-Cooperation, Collaborative project, Small or medium-scale focused research project, FP7-SPACE-2013-1 scheme, Grant Agreement
Number 607254.

[Acknowledgements](http://herschel.sussex.ac.uk/acknowledgements)